In [1]:
from langchain import PromptTemplate # with this we can create a prompt
from langchain.chains import RetrievalQA # with this we can ask questions to the model
from langchain.embeddings import HuggingFaceEmbeddings # with this we can convert text to vector
from langchain.vectorstores import Pinecone # with this we can store the vector
import pinecone 
from langchain.document_loaders import PyPDFLoader,JSONLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # With this we can split the text into smaller chunks
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers   
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

c:\Users\us\anaconda3\envs\mindbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
# # Load pdf and then convert it to JSON
# import json
# from langchain.document_loaders import DirectoryLoader

# def load_pdf(data):
#     loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
#     documents = []
#     for pdf_file in loader.load():
#         pdf_data = {}
#         pdf_data['main_query'] = pdf_file.metadata.get('main_query', '')  # Access metadata directly
#         pdf_data['context_queries'] = [pdf_file.page_content]  # Access page content directly
#         documents.append(pdf_data)
#     return json.dumps(documents)
# extracted_data = load_pdf("E:/Code/TherapyTalk-Chatbot-using-Llama2/data/")




In [2]:
# Extract data from pdf
def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [2]:
# # Extract data from json
# def load_json(data, jq_schema):
#     loader = DirectoryLoader(data,
#                              glob="*.json",
#                              loader_cls=JSONLoader,
#                              loader_kwargs={'jq_schema': jq_schema})
    
#     documents = loader.load()
#     return documents

In [15]:
# import subprocess

# def run_jq(command):
#     jq_path = r'C:\jq\jq\jq-win64.exe'  # Ensure the correct path
#     process = subprocess.Popen([jq_path, '-r', command], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     output, error = process.communicate()
#     if process.returncode != 0:
#         raise Exception(f'jq command failed with error: {error.decode()}')
#     return output.decode()

# # get output
# jq_schema = '{"main_query": .main_query, "context_queries": .context_queries}'
# output = run_jq(jq_schema)

# print(output)

In [3]:
# #Create subprocess to run jq
# import subprocess

# def run_jq(command,json_file):
#     jq_path = 'C:\jq\jq\jq-win64.exe'  # Replace with the actual path to your jq executable
#     process = subprocess.Popen([jq_path, '-r', command], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     output, error = process.communicate()
#     if process.returncode != 0:
#         raise Exception(f'jq command failed with error: {error.decode()}')
#     return output.decode()


In [4]:
# # Construct the jq command to extract main_query and context_queries
# jq_command = '.main_query, .context_queries'
# json_file = 'data/test.json'  # Path to your JSON file

In [6]:
# # Run the jq command using the run_jq function
# try:
#     output = run_jq(jq_command, json_file)
#     # print(output)
# except Exception as e:
#     print(f"Error: {e}")

In [3]:
extracted_data= load_pdf("data/")

In [5]:
extracted_data[1]

Document(page_content="wants to know what it thinks is most important to them. It's a race between Facebook and Google to\nfigure it out. One minute your brand could be popping up at the top of a user’s page; the next it could be\nburied six pages down. Facebook may decide that sharing is a much stronger call to action and\nbrandendorsement than liking. If your content happens to elicit many shares, you’re golden. But then\nFacebook could change its mind and decide that likes are actually as valuable if not more so than\nshares. Your content doesn’t usually get that many likes. Now what? The key to great marketing is\nremembering that even though you’re all about your brand, your customer is not. As with any first date,\ngetting a second date depends on you doing your best to learn more about what the other person is\ninterested in. In the end, boxing and dating are really not that different. After all, the goal is to score.\nSometimes the score is measured in points, and sometimes in 

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [7]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

402


In [8]:
# download the embedding model
def download_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_embedding_model()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [11]:
query_results = embeddings.embed_query("Hello World")
print("length", len(query_results))

length 384


In [12]:
# Initialize pinecone
from pinecone import Pinecone
PINECONE_API_KEY = "e5b0b3f3-d5c3-4f55-8053-976ad38419f6"
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("mindbot")

In [13]:
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'mindbot-caeac3f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'mindbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [15]:
# Creating embeddings for each of the text chunks and store them in pinecone. (This is our knowledge base) 
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], index_name='mindbot', embedding=embeddings)

In [17]:
# If we already have an index we can load it (This is also checking ranked results)
docsearch = PineconeVectorStore.from_existing_index( index_name='mindbot', embedding=embeddings) # with this we can load the vector

query = "What is digital marketing"

docs = docsearch.similarity_search(query, k=3)# here k is the number of results we want


In [18]:
print(docs)

[Document(page_content="managers hoping to benefit from digital content marketing, we advocate further investments into\ntechnologies that improve the selling firm’s ability to target content based on the customer■'s\nidiosyncratic needs at different journey stages. Gartner found that, when considering a relevant content"), Document(page_content='marketing is the marketing of a business or brand through the sharing of educational, entertaining, or\ninsightful information that will help consumers enrich their lives. It is not about forcing a sales pitch on\nconsumers but about helping them move towards the best option for themselves (which may or may not\nbe buying the marketers brand) (Ruffolo 2017.) There are many ways customer value can be created.'), Document(page_content='literature strongly posits brand or brand-related content, this is not always the best way to go about\ndeveloping a digital content distribution strategy. This is the first study to demonstrate the positive effec

In [19]:
# Defining Prompt Template
promp_template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know. Do not make up an answer.
context: {context}
question: {question}
Only return the helpful answer and information about the context. Do not make up an answer.
Helpful Answer:
"""

In [20]:
# Creating Prompt Template
prompt = PromptTemplate(template=promp_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": prompt} # Created this chain type argument because I will be using Retrieval QA chain

In [21]:
# Loading llm model
llm = CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens": 512,
                            "temperature": 0.8})

### Explaining Retriever Class below:

Importing necessary modules and classes:
Python
```
from langchain.schema.retriever import BaseRetriever
from typing import List
from langchain.schema import Document
from pydantic import BaseModel
```
These lines import the necessary modules and classes. BaseRetriever is the base class for all retrievers, List is a typing construct that denotes a list, Document is a class that represents a document, and BaseModel is a base class for Pydantic models.

<b>Defining the PineconeRetriever class:</b>
```
class PineconeRetriever(BaseRetriever, BaseModel):
    pinecone_vector_store: PineconeVectorStore
```

This line defines the PineconeRetriever class which inherits from both BaseRetriever and BaseModel. It has a single attribute pinecone_vector_store of type PineconeVectorStore.

<b>Defining the constructor:</b>
```
def __init__(self, pinecone_vector_store: PineconeVectorStore, **data):
    super().__init__(pinecone_vector_store=pinecone_vector_store, **data)
```
The constructor takes two arguments: pinecone_vector_store and **data. It calls the constructor of the superclass with these arguments.

<b>Defining the _get_relevant_documents method:</b>
```

def _get_relevant_documents(self, query: str) -> List[Document]:
    return self.pinecone_vector_store.similarity_search(query)
```
This method takes a query string and returns a list of Document objects that are relevant to the query. It does this by calling the similarity_search method of the pinecone_vector_store object.

<b>Defining the _aget_relevant_documents method:</b>
```
async def _aget_relevant_documents(self, query: str) -> List[Document]:
    return self.pinecone_vector_store.similarity_search(query)
```
This method is the asynchronous version of _get_relevant_documents. It does the same thing but in an asynchronous manner. This can be useful when dealing with IO-bound tasks.

In [22]:
# Defining retriever
from langchain.schema.retriever import BaseRetriever
from typing import List
from langchain.schema import Document
from pydantic import BaseModel

class PineconeRetriever(BaseRetriever, BaseModel):
    pinecone_vector_store: PineconeVectorStore

    def __init__(self, pinecone_vector_store: PineconeVectorStore, **data):
        super().__init__(pinecone_vector_store=pinecone_vector_store, **data)

    def _get_relevant_documents(self, query: str) -> List[Document]:
        return self.pinecone_vector_store.similarity_search(query)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        return self.pinecone_vector_store.similarity_search(query)


In [23]:
# Create a PineconeRetriever object
pinecone_retriever = PineconeRetriever(docsearch)

In [24]:
# Creating Retrieval QA  Object
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pinecone_retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [25]:
while True:
    user_input = input(f"Ask your question: ")
    if query == "exit":
        break
    else:
        result = qa({"query": user_input})
        print(result)

{'query': 'Hi What is SEO', 'result': "The answer question. \nI don's on brand- Content Marketing and informational\nYou can be removedhelpfulfill in content marketing the content on consumers are used to make-Google Ping search engine optimization,\ncontent marketing and Google Pandaid=\nI do not helpful answer is a content marketing customer rewards competitive vs. 0\nIf you have an empirical or not helpfulAnswer: YesHelpfulness, Patelcome in the impact of keywords and Content marketing and content marketing a systematic content marketing\nNo help@\nContent Marketing on social media use of content marketing your answer.  I can help\nA\nTo improve\nContent Marketers for customers’\nGoogle's on customer experience, but do not helpful answer to get outstanding\nContent marketing relevant (I don' Content quality and social media usage and quality on the question- You can provide your SEOurseofficacy and informative content. (TrueHelpful Answer:\nThe answer_ Do not helpful  A case of this